<a href="https://colab.research.google.com/github/mertyyanik/Natural-Language-Processing-Real-or-Fake/blob/master/News_Tensorflow2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
 
from pydrive.drive import GoogleDrive
 
from google.colab import auth
 
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
 
gauth = GoogleAuth()
 
gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.6MB/s 


In [0]:
from google.colab import files

#Test.csv Dosyası

file_id = '1Zx7P0t9lQLP4aj4YOUaZDxNRrSkZOTk4'

downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('test.csv')

#Test.txt Dosyası
 
file_id = '1S2-N2HSIgtZpSzPmAEZWmVxV2YlOzCzs'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('test.txt')

#clean_real-Train.txt Dosyası
 
file_id = '1pnsf7izWzNYlA41xcWrCXQ5SSFe5JCUF'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('clean_real-Train.txt')

#clean_fake-Train.txt Dosyası
 
file_id = '1Mc1m6S6WCJuRkMBWM9_LZGHSzJIw2-e0'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('clean_fake-Train.txt')


In [0]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score
import time

In [0]:
# Reading Data
input1Data = open('clean_real-Train.txt', 'r+')
input2Data = open('clean_fake-Train.txt', 'r+')
testData = open('test.txt')

testDataForComparison = pd.read_csv('test.csv').iloc[:, 1:]

input1List = []
input2List = []
testList = []

read = input1Data.readline()
read2 = input2Data.readline()
read3 = testData.readline()
rowCount = 0
while read != "":
    input1List.append(read)
    read = input1Data.readline()
    
while read2 != "":
    input2List.append(read2)
    read2 = input2Data.readline()

while read3 != "":
    testList.append(read3)
    read3 = testData.readline()

In [5]:
# Real and Fake data combined.    
liste = ['real' for i in range(len(input1List))]
realInput = np.column_stack((input1List, liste))


liste = ['fake' for i in range(len(input2List))]
fakeInput = np.column_stack((input2List, liste))

mainInput = realInput.tolist() + fakeInput.tolist()
random.shuffle(mainInput)

sentenceInput = []
targetInput = []
targetInputFloat = []
for i in range(len(mainInput)):
    sentenceInput.append(mainInput[i][0])
    
for i in range(len(mainInput)):
    targetInput.append(mainInput[i][1])
    
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
targetInputFloat = lb.fit_transform(targetInput)
testDataForComparison = lb.fit_transform(testDataForComparison)

targetInputFloat = pd.DataFrame(data=targetInputFloat)
testDataForComparison = pd.DataFrame(data = testDataForComparison)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
# Stop Words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stemmer
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# Regular Expressions
import re

sentenceList = []
for i in range(len(sentenceInput)):
    # Regular Expressions
    comment = re.sub('[^a-zA-Z]', ' ', sentenceInput[i]) #-> 1.Adım
    # Upper Case Problem
    comment = comment.lower() #-> 2. Adım
    # Splitting
    comment = comment.split() #-> 3.Adım
    # Stemming and Stop Words
    comment = [ps.stem(kelime) for kelime in comment if not kelime in set(stopwords.words('english'))] # -> 4. Adım
    
    comment = ' '.join(comment)
    sentenceList.append(comment)
    
temporaryResult = pd.DataFrame(data=sentenceList)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# Vectorized All Data
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentenceList)

result = pd.DataFrame(data=X.toarray())


In [0]:
# Test Data
testSentenceList = []
for i in range(len(testList)):
    # Regular Expressions
    comment = re.sub('[^a-zA-Z]', ' ', testList[i])
    # Upper Case Problem
    comment = comment.lower()
    # Splitting
    comment = comment.split()
    # Stemming and Stop Words
    comment = [ps.stem(kelime) for kelime in comment if not kelime in set(stopwords.words('english'))]
    
    comment = ' '.join(comment)
    testSentenceList.append(comment)

X = vectorizer.transform(testSentenceList)

testResult = pd.DataFrame(data=X.toarray())


In [0]:
!pip install tensorflow==2.0.0-beta1 

In [0]:
import tensorflow as tf

def create_dataset(x, y, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  #dataset = dataset.shuffle(len(x))
  dataset = dataset.batch(batch_size)
  return dataset

train_set = create_dataset(result.values, targetInputFloat.values, len(result))
test_set = create_dataset(testResult.values, testDataForComparison.values, len(testResult))

In [0]:
features, labels = next(iter(train_set))
print(labels)

In [0]:
import tensorflow as tf

model = tf.keras.Sequential([
                         tf.keras.layers.Dense(1500, kernel_initializer='uniform', activation=tf.nn.relu, input_dim=3854),
                         tf.keras.layers.Dropout(0.5),
                         tf.keras.layers.Dense(1500, kernel_initializer='uniform', activation=tf.nn.relu),
                         tf.keras.layers.Dropout(0.5),
                         tf.keras.layers.Dense(1, kernel_initializer = 'uniform', activation =tf.nn.sigmoid),
])

In [0]:
def loss_function(model, observation, label):
  prediction = model(observation, training = True)
  loss = tf.keras.losses.binary_crossentropy(label, prediction)
  return loss

for gozlem, etiket in train_set.take(1):
  print(f"Gerçeğe olan uzaklık: {loss_function(model, gozlem, etiket)}")
  

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
def gradient_process(model, observation, label):
    
    with tf.GradientTape() as tape:
        loss = loss_function(model, observation, label)
        
    gradients_of_model = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients_of_model, model.trainable_variables))
    return loss

In [0]:
num_epochs = 40

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()

  for x, y in train_set:
    loss_value = gradient_process(model, x, y)

    epoch_loss_avg(loss_value)  
    
  if epoch % 2 == 0:
    print(f'Epoch {epoch}: Loss: {epoch_loss_avg.result()}')

In [0]:
import numpy as np
prediction = []
for x, y in test_set:
  prediction.append(model.predict(x))

prediction = np.asarray(prediction)

prediction = (prediction > 0.5)

prediction = prediction.reshape(489,)

In [0]:
from sklearn.metrics import accuracy_score

acScoreNN = accuracy_score(testDataForComparison, prediction)
print("Accuracy : " + str(acScoreNN))